In [ ]:
from openai import OpenAI

from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy import stats
import json
from itertools import combinations
import time
import random
import os
from dotenv import load_dotenv

We are trying to establish what could be going wrong with the docker container by breaking the functions in my module apart then testing them seperatly. After that we will test the fast api call and make sure no errors are still occuring. Finally the corret functions and api call will be put into a docker container

In [ ]:
# Load the environment variables from .env file
#use the .env file to not expose our secret API key
#load_dotenv()
#client = OpenAI(
    # This is the default and can be omitted
#    api_key=os.environ.get("OPENAI_API_KEY"),
#)

In [ ]:
dotenv_path = os.path.join(os.path.dirname('lim-stress-tes'), '.env')
load_dotenv(dotenv_path)
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

The first function is called get_openai_responses. It takes in a prompt and will obtain responses. The number of responses can be controlled by the num_responses. To limit the tokens set the varaible max_token. Initial_delay,exponetial_base are used when a rate limit occurs and it emplements exponetial backing. The function is set to retry 10 times if a rate limit error occurs. Adjust if need be

***to test I set the num_responses = 5

In [ ]:
def get_openai_responses(prompt, num_responses=40, max_tokens=100, initial_delay: float = 1,
                         exponential_base: float = 2, jitter: bool = True):
    
    """
    Retrieves a specified number of responses from OpenAI's GPT model.
    
    Args:
        prompt (str): The prompt to send to the model.
        num_responses (int): The number of responses to retrieve.
        max_tokens (int): The maximum number of tokens per response.
        initial_delay (float): Initial delay in seconds before retrying after a rate limit error.
        exponential_base (float): The base for the exponential backoff calculation.
        jitter (bool): Whether to add random jitter to the delay.

    Returns:
        list: A list of responses from the AI model.
    """
    if num_responses > 100:  # Example threshold
        raise ValueError("num_responses is too high. Please reduce the number.")

    delay = initial_delay
    responses = []
    retries = 0
    max_retries = 10

    while len(responses) < num_responses and retries < max_retries:
        try:
            AI_response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that provides recommendations."},
                    {"role": "user", "content": prompt},
                ],
                max_tokens=max_tokens,
                n=1,  # Number of completions to generate
            )
            #print(AI_response)
            #need to eventually un comment these
            response_content = AI_response.choices[0].message.content
            responses.append(response_content)
        except Exception as e:
            print(f"Encountered an error: {e}")
            if 'rate limit' in str(e).lower():
                delay *= exponential_base * (1 + jitter * random.random())
                print(f"Rate limit exceeded. Waiting for {delay} seconds.")
                time.sleep(delay)
            else:
                print("Encountered a non-rate-limit error. Retrying...")
            retries += 1
            time.sleep(initial_delay)  # Basic delay for non-rate-limit errors

    if retries == max_retries:
        print("Max retries reached. Some responses may not have been retrieved.")

    return responses

The next function is called get_bert_embedding. As the name states this function will obtain the bertembeddings 

In [ ]:
def get_bert_embedding(my_text):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')
    tokens = tokenizer(my_text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**tokens)
    last_hidden_states = outputs.last_hidden_state
    embedding = torch.mean(last_hidden_states, dim=1)
    
    # Convert to numpy array and check shape
    embedding_np = embedding.numpy()
    #print(f"Embedding shape: {embedding_np.shape}")  # For debugging
    return embedding_np.tolist()

The last function in my module is process_prompt. This function will compare the cosine similarities for each response and compare each response to everyother response created. It will then return the quant scores

In [ ]:
def process_prompt(prompt):
    result_data = []  # Store quant scores for all prompts
    responses = get_openai_responses(prompt)

    prompt_embedding = get_bert_embedding(prompt)
    response_embeddings = [get_bert_embedding(response) for response in responses]

    cosine_sim_scores = []
    embedding_pairs = list(combinations([prompt_embedding] + response_embeddings, 2))
    print(embedding_pairs)
    for pair in embedding_pairs:
        score = cosine_similarity([pair[0]], [pair[1]])[0][0]
        cosine_sim_scores.append(score)

    mean_score = np.mean(cosine_sim_scores)
    median_score = np.median(cosine_sim_scores)
    mode_score = stats.mode(cosine_sim_scores)
    average_score = np.average(cosine_sim_scores)
    sample_size = len(cosine_sim_scores)

    quant_scores = {
        "mean": mean_score,
        "median": median_score,
        "mode": mode_score,
        "average": average_score,
        "sample_size": sample_size
    }

    result_data.append({"prompt": prompt, "quant_scores": quant_scores})

    return result_data

Now we will test if we can get replies from the three functions in the module

In [ ]:
#Test Cell For get_openai_responses
my_prompt='What is today?'
my_results=get_openai_responses(my_prompt, num_responses=5)
print(my_prompt)
print(my_results)

In [ ]:
#Test for berkembeding function:
my_result2=get_bert_embedding(my_results[0])
print(my_result2)

In [ ]:
#Test Cell
my_prompt='What is today?'
my_results3=process_prompt(my_prompt)
print(my_prompt)
print(my_results3)